In [ ]:
!apt update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar -xvf spark-3.3.0-bin-hadoop3.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
47 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as 

In [ ]:
!pip install underthesea
!pip install pyvi
!pip install langdetect

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
file_path = '/content/gdrive/MyDrive/LDS0_k296_NgoTrieuPhu/Project_1/Data_Agoda_raw/hotel_comments.csv'

In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, StopWordsRemover, RegexTokenizer
from pyspark.ml.feature import CountVectorizer, IDF, StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import IntegerType

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import DecisionTreeClassifier

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
import pandas as pd
import numpy as np
import regex
import re
import string

from underthesea import word_tokenize, pos_tag, sent_tokenize, sentiment
from langdetect import detect

In [ ]:
spark = SparkSession.builder.appName('pj_1').getOrCreate()

## Đọc dữ liệu

In [ ]:
data = spark.read.csv(file_path, inferSchema=True, header=True)
data.count()

80314

In [ ]:
data.printSchema()

root
 |-- num: integer (nullable = true)
 |-- Hotel ID: string (nullable = true)
 |-- Reviewer ID: string (nullable = true)
 |-- Reviewer Name: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Group Name: string (nullable = true)
 |-- Room Type: string (nullable = true)
 |-- Stay Details: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Score Level: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Body: string (nullable = true)
 |-- Review Date: string (nullable = true)



In [ ]:
data.select([count(when(isnan(c),c)).alias(c) for c in data.columns]).toPandas().T

,0
num,0
Hotel ID,0
Reviewer ID,0
Reviewer Name,3
Nationality,0
Group Name,0
Room Type,0
Stay Details,0
Score,0
Score Level,0


In [ ]:
data.select([count(when(isnull(c),c)).alias(c) for c in data.columns]).toPandas().T

,0
num,0
Hotel ID,0
Reviewer ID,0
Reviewer Name,86
Nationality,0
Group Name,0
Room Type,0
Stay Details,0
Score,0
Score Level,0


In [ ]:
data = data.dropDuplicates()
data.count()

80314

In [ ]:
data.show(3)

+---+--------+-----------+-------------+-----------+------------------+--------------------+--------------------+-----+-----------------+------------------+--------------------+--------------------+
|num|Hotel ID|Reviewer ID|Reviewer Name|Nationality|        Group Name|           Room Type|        Stay Details|Score|      Score Level|             Title|                Body|         Review Date|
+---+--------+-----------+-------------+-----------+------------------+--------------------+--------------------+-----+-----------------+------------------+--------------------+--------------------+
|402|     1_1|    1_1_402|         Ynhu|   Việt Nam|  Du lịch một mình|Phòng Gia Đình Lo...|Đã ở 2 đêm vào Th...| 10,0|Trên cả tuyệt vời|Trên cả tuyệt vời”|                Good|Đã nhận xét vào 0...|
|423|     1_1|    1_1_423|         Hong|   Việt Nam|Gia đình có trẻ em|     Executive Suite|Đã ở 1 đêm vào Th...| 10,0|Trên cả tuyệt vời|Trên cả tuyệt vời”|Khách sạn đẹp nhâ...|Đã nhận xét vào 1...|
|659|

In [ ]:
data.groupBy('Nationality').count().orderBy('count', ascending = False).show()

+--------------+-----+
|   Nationality|count|
+--------------+-----+
|      Việt Nam|49433|
|      Hàn Quốc|14441|
|        Hoa Kỳ| 2324|
|            Úc| 1746|
|      Malaysia| 1653|
|     Hồng Kông| 1528|
|    Trung Quốc| 1025|
|      Thái Lan|  995|
|Vương quốc Anh|  916|
|        Canada|  873|
|     Singapore|  842|
|      Nhật Bản|  532|
|           Đức|  419|
|          Pháp|  407|
|         Ấn Độ|  330|
|           Nga|  321|
| Bắc Macedonia|  252|
|      Đài Loan|  241|
|   New Zealand|  225|
|   Philippines|  153|
+--------------+-----+
only showing top 20 rows



In [ ]:
data.groupBy('Group Name').count().orderBy('count', ascending = False).show()

+------------------+-----+
|        Group Name|count|
+------------------+-----+
|           Cặp đôi|31068|
| Gia đình có em bé|20276|
|              Nhóm|10773|
|  Du lịch một mình| 8943|
|Gia đình có trẻ em| 6351|
|       Đi công tác| 2903|
+------------------+-----+



In [ ]:
data.groupBy('Score Level').count().orderBy('count', ascending = False).show()

+-----------------+-----+
|      Score Level|count|
+-----------------+-----+
|Trên cả tuyệt vời|58051|
|        Tuyệt vời|14814|
|          Rất tốt| 4356|
|         Hài Lòng| 3093|
+-----------------+-----+



## Tiền xử lý dữ liệu

### Lọc ra các dòng dữ liệu tiếng Việt trong Body

In [ ]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"
detect_language_udf = udf(detect_language, StringType())
t = data.withColumn('Language', detect_language_udf(data['Body']))
vietnamese_comments_df = t.filter(t['Language'] == 'vi')
vietnamese_comments_df.show()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=03efacc044c2c83f181298c2ab9286d264d1b960ed68e6d3848d5818bdf67856
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
+----+--------+-----------+-------------+-----------+------------------+--------------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+--------+
| num|Hotel ID|Reviewer ID|Reviewer Name|Nationality|        Group Name|           Room Type|        Stay Details|Score|      Score Level|               Title|                Body|         Review Date|Language|
+----+--------+-----------+-------------+-----------+------------------+--------------------+--------------------+-----+-----------------+--------------------

In [ ]:
##LOAD EMOJICON
file = open('/content/gdrive/MyDrive/LDS0_k296_NgoTrieuPhu/Project_1/files/emojicon.txt', 'r', encoding="utf8")
emoji_lst = file.read().split('\n')
emoji_dict = {}
for line in emoji_lst:
    key, value = line.split('\t')
    emoji_dict[key] = str(value)
file.close()
#################
#LOAD TEENCODE
file = open('/content/gdrive/MyDrive/LDS0_k296_NgoTrieuPhu/Project_1/files/teencode.txt', 'r', encoding="utf8")
teen_lst = file.read().split('\n')
teen_dict = {}
for line in teen_lst:
    key, value = line.split('\t')
    teen_dict[key] = str(value)
file.close()
###############
#LOAD TRANSLATE ENGLISH -> VNMESE
file = open('/content/gdrive/MyDrive/LDS0_k296_NgoTrieuPhu/Project_1/files/english-vnmese.txt', 'r', encoding="utf8")
english_lst = file.read().split('\n')
english_dict = {}
for line in english_lst:
    key, value = line.split('\t')
    english_dict[key] = str(value)
file.close()
################
#LOAD wrong words
file = open('/content/gdrive/MyDrive/LDS0_k296_NgoTrieuPhu/Project_1/files/wrong-word.txt', 'r', encoding="utf8")
wrong_lst = file.read().split('\n')
file.close()
#################
#LOAD STOPWORDS
file = open('/content/gdrive/MyDrive/LDS0_k296_NgoTrieuPhu/Project_1/files/vietnamese-stopwords.txt', 'r', encoding="utf8")
stopwords_lst = file.read().split('\n')
file.close()

### Xử lý dữ liệu tiếng Việt

In [ ]:
def process_text(text, emoji_dict, teen_dict, wrong_lst):
    document = text.lower()
    document = document.replace("’",'')
    document = regex.sub(r'\.+', ".", document)
    new_sentence =''
    for sentence in sent_tokenize(document):
        # if not(sentence.isascii()):
        ###### DEL Punctuation & Numbers
        pattern = r'(?i)\b[a-záàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ]+\b'
        sentence = ' '.join(regex.findall(pattern,sentence))
        ###### CONVERT EMOJICON
        sentence = ''.join(emoji_dict[word]+' ' if word in emoji_dict else word for word in list(sentence))
        ###### CONVERT TEENCODE
        sentence = ' '.join(teen_dict[word] if word in teen_dict else word for word in sentence.split())

        # ...
        ###### DEL wrong words
        sentence = ' '.join('' if word in wrong_lst else word for word in sentence.split())
        new_sentence = new_sentence+ sentence + '. '
    document = new_sentence
    #print(document)
    ###### DEL excess blank space
    document = regex.sub(r'\s+', ' ', document).strip()
    #...
    return document

In [ ]:
process_text_spark_udf = udf(lambda text: process_text(text, emoji_dict, teen_dict, wrong_lst), StringType())

In [ ]:
tmp = vietnamese_comments_df.drop('num', 'Reviewer ID', 'Reviewer Name')
data_pre = tmp.withColumn('Body_pre', process_text_spark_udf(col('Body')))
data_pre.show(30,truncate=False)

+--------+-----------+------------------+-------------------------------------------------------------+--------------------------------+-----+-----------------+------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Hotel ID|Nationality|Group Name        |Room Type                                                    |Stay Details       

In [ ]:
# Chuẩn hóa unicode tiếng việt
def loaddicchar():
    uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
    unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic

# Đưa toàn bộ dữ liệu qua hàm này để chuẩn hóa lại
def covert_unicode(txt):
    dicchar = loaddicchar()
    return regex.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

In [ ]:
covert_unicode_udf = udf(lambda text: covert_unicode(text), StringType())

In [ ]:
data_pre = data_pre.withColumn('Body_pre_load', covert_unicode_udf(col('Body_pre')))
data_pre.show(truncate=False)

In [ ]:
def process_special_word(text):
    # có thể có nhiều từ đặc biệt cần ráp lại với nhau
    new_text = ''
    text_lst = text.split()
    i= 0
    # không, chẳng, chả...
    if 'không' in text_lst:
        while i <= len(text_lst) - 1:
            word = text_lst[i]
            #print(word)
            #print(i)
            if  word == 'không':
                next_idx = i+1
                if next_idx <= len(text_lst) -1:
                    word = word +'_'+ text_lst[next_idx]
                i= next_idx + 1
            else:
                i = i+1
            new_text = new_text + word + ' '
    else:
        new_text = text
    return new_text.strip()

In [ ]:
process_special_word_udf = udf(lambda text: process_special_word(text), StringType())

In [ ]:
data_pre = data_pre.withColumn('Body_pre_fea', process_special_word_udf(col('Body_pre_load')))
data_pre.show(truncate=False)

+--------+-----------+------------------+--------------------------------------------------+-------------------------------+-----+-----------------+--------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
def normalize_repeated_characters(text):
    # Thay thế mọi ký tự lặp liên tiếp bằng một ký tự đó
    # Ví dụ: "lònggggg" thành "lòng", "thiệtttt" thành "thiệt"
    return re.sub(r'(.)\1+', r'\1', text)

In [ ]:
normalize_repeated_characters_udf = udf(lambda text: normalize_repeated_characters(text), StringType())

In [ ]:
data_pre = data_pre.withColumn('Body_pre_fea_normi', normalize_repeated_characters_udf(col('Body_pre_fea')))
data_pre.show(truncate=False)

+--------+-----------+------------------+--------------------------------------------------+-------------------------------+-----+-----------------+--------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
def process_postag_thesea(text):
    new_document = ''
    # text = text.encode('utf-8')
    for sentence in sent_tokenize(text):
        # if isinstance(sentence, bytes):
        #     sentence = sentence.decode('utf-8')
        sentence = sentence.replace('.', '')
        ###### POS tag
        lst_word_type = ['N', 'Np', 'A', 'AB', 'V', 'VB', 'VY', 'R']
        tokens = word_tokenize(sentence, format="text")
        pos_tags = pos_tag(process_special_word(tokens))
        print(pos_tags)
        sentence = ' '.join(word[0] if word[1].upper() in lst_word_type else '' for word in pos_tags)
        new_document += sentence + ' '
    ###### DEL excess blank space
    new_document = regex.sub(r'\s+', ' ', new_document).strip()
    return new_document

In [ ]:
process_postag_thesea_udf = udf(process_postag_thesea, StringType())

In [ ]:
data_pre = data_pre.withColumn('Body_pre_fea_precess', process_postag_thesea_udf(col('Body_pre_fea_normi')))
data_pre.show(truncate = False)

+--------+-----------+------------------+--------------------------------------------------+-------------------------------+-----+-----------------+--------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
def remove_stopword(text, stopwords = stopwords_lst):
    ###### REMOVE stop words
    document = ' '.join('' if word in stopwords else word for word in text.split())
    ###### DEL excess blank space
    document = regex.sub(r'\s+', ' ', document).strip()
    return document

In [ ]:
remove_stopword_udf = udf(remove_stopword, StringType())

In [ ]:
data_pre = data_pre.withColumn('Body_pre_fea_remove', remove_stopword_udf(col('Body_pre_fea_precess')))
data_pre.show(truncate = False)

+--------+-----------+------------------+--------------------------------------------------+-------------------------------+-----+-----------------+--------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Lưu data sau khi xử lý và tiếp tục ở file pyspark_2(gc).ipynb

In [ ]:
data_pre.drop('Score Level', 'Title', 'Language', 'Body').write.csv('/content/gdrive/MyDrive/Đồ_án/Projects_1/Data_Agoda_raw/data_new', header=True, sep = ',')